### 简单模型由复杂模型转化而来

这是最简单的torch的神经网络模型

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from PIL import Image

# 自定义Dataset类
class CustomImageDataset(Dataset):
    def __init__(self, image_dir, transform=None):
        """
        初始化方法
        :param image_dir: 图像存放的目录路径
        :param transform: 数据预处理
        """
        self.image_dir = image_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []
        
        # 遍历所有类别文件夹
        for label, category in enumerate(os.listdir(image_dir)):
            category_path = os.path.join(image_dir, category)
            if os.path.isdir(category_path):
                for img_name in os.listdir(category_path):
                    img_path = os.path.join(category_path, img_name)
                    self.image_paths.append(img_path)
                    self.labels.append(label)
        
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        """
        获取指定索引的图像及其标签
        :param idx: 索引
        :return: 图像和标签
        """
        img_path = self.image_paths[idx]
        label = self.labels[idx]
        image = Image.open(img_path).convert("RGB")  # 打开图像

        if self.transform:
            image = self.transform(image)  # 应用预处理
        
        return image, label

# 设置数据预处理
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# 加载数据集
train_dir = 'path_to_your_dataset'  # 设置图像数据所在的目录路径
train_dataset = CustomImageDataset(image_dir=train_dir, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# 定义简单的卷积神经网络模型
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(64*32*32, 128)
        self.fc2 = nn.Linear(128, 10)  # 假设有10个类别
        self.max_pool = nn.MaxPool2d(2)
    
    def forward(self, x):
        x = self.max_pool(torch.relu(self.conv1(x)))
        x = self.max_pool(torch.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)  # 展平
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# 初始化模型、损失函数和优化器
model = SimpleCNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 训练模型
for epoch in range(10):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()  # 清空梯度
        outputs = model(inputs)  # 前向传播
        loss = criterion(outputs, labels)  # 计算损失
        loss.backward()  # 反向传播
        optimizer.step()  # 优化参数
        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/10], Loss: {running_loss/len(train_loader)}")

### mmcv和mmengine是怎么来的

为了能够直接通过配置文件进行更改，而且兼容各种不同的预处理方法和模型，从torch抽象出关键几个步骤出来

将模型代码变得能够兼容各种方法，且训练过程简洁

In [ ]:
from mmengine import Config
from mmcv import Config
from mmengine.runner import Runner
from mmengine.datasets import build_dataset
from mmengine.models import build_model
from mmengine.optimizers import build_optimizer
from mmengine.utils import setup_logger

# 配置文件（config）
cfg = Config.fromfile('./configs/simplenet_config.py')

# 构建数据集
dataset = build_dataset(cfg.data.train)

# 构建模型
model = build_model(cfg.model)

# 构建优化器
optimizer = build_optimizer(model, cfg.optimizer)

# 配置日志
logger = setup_logger()

# 创建训练器
runner = Runner(model, optimizer, cfg, logger=logger)

# 训练模型
runner.train()

In [ ]:
# configs/simplenet_config.py

# 模型配置
model = dict(
    type='SimpleNN',  # 这里指定我们使用的模型类型
    input_size=(224, 224, 3),  # 假设输入图片的尺寸
    output_size=1,  # 假设这是回归任务，输出是一个标量
    layers=dict(
        fc1=dict(in_features=224*224*3, out_features=512),
        fc2=dict(in_features=512, out_features=1)
    )
)

# 数据配置
data = dict(
    train=dict(
        type='CustomDataset',  # 数据集类型
        data_dir='path_to_images',  # 训练图片路径
        annotations_dir='path_to_annotations',  # 训练标注路径
        batch_size=1,  # 每次训练的样本数
        shuffle=True,  # 是否打乱数据
        transform=[
            dict(type='Resize', size=(224, 224)),  # 数据预处理：调整图片大小
            dict(type='ToTensor'),  # 转换为Tensor格式
            dict(type='Normalize', mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # 归一化
        ]
    )
)

# 优化器配置
optimizer = dict(
    type='SGD',  # 使用SGD优化器
    lr=0.01,  # 学习率
    momentum=0.9  # 动量
)

# 损失函数配置
criterion = dict(
    type='MSELoss'  # 使用均方误差损失函数
)

# 训练配置
train_cfg = dict(
    epochs=10,  # 训练轮数
    log_interval=10  # 每10步打印一次日志
)

# 日志和输出配置
logger = dict(
    type='TensorBoardLogger',  # 使用TensorBoard记录日志
    log_dir='./logs'  # 日志存放路径
)

### openmmlab养成

不断丰富各种框架与模型，兼容各种功能，就形成了现在的openmmlab